## Рождественский Владислав P4309

## Установка Библиотек

In [110]:
%pip install --upgrade --quiet yandex-cloud-ml-sdk

Note: you may need to restart the kernel to use updated packages.


Установка markdown для ответов бота

In [111]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

## Авторизация в YandexCloud

Чтобы получить необходимые токены необходимо авторизоваться в Yandex Cloud, подключить карту для платежей, создать сервисный аккаунт со всеми доступами и скопировать folder_id и api_key, все значения необходимо вставить в .env файл

In [112]:
from dotenv import load_dotenv, dotenv_values
load_dotenv()

folder_id = dotenv_values()['folder_id']
api_key = dotenv_values()['api_key']

Создаем модель на основе yandex gpt

In [113]:
import os
from yandex_cloud_ml_sdk import YCloudML

sdk = YCloudML(folder_id=folder_id, auth=api_key)

# sdk.setup_default_logging(log_level='DEBUG')

model = sdk.models.completions("yandexgpt", model_version='rc')

Отправим тестовый запрос

In [114]:
printx(model.run("Какое вино можно пить со стейком?").text)

Выбор вина к стейку зависит от степени прожарки, вида мяса и личных предпочтений. Вот несколько общих рекомендаций:

1. **Для стейка из говядины:**
* **Красное вино:** стейки из говядины хорошо сочетаются с красными винами, такими как каберне совиньон, шираз (сира), мерло, мальбек или зинфандель. Эти вина обычно имеют насыщенный вкус, танинность и хорошую структуру, которые могут дополнить жирность и вкус мяса.
* **Танины:** танины в вине помогают сбалансировать жирность стейка, создавая гармоничное сочетание.

2. **В зависимости от степени прожарки:**
* **Слабая прожарка (rare):** для более нежных стейков с кровью могут подойти лёгкие и фруктовые красные вина, которые не будут перебивать тонкий вкус мяса.
* **Средняя прожарка (medium):** для стейков средней прожарки можно выбрать вина средней плотности и структуры, которые подчеркнут вкус мяса, не доминируя над ним.
* **Хорошо прожаренный стейк (well-done):** для более «тяжёлых» и насыщенных стейков подойдут более крепкие и танинные вина.

3. **Другие варианты:**
* В некоторых случаях можно экспериментировать с розовыми или даже белыми винами, особенно если стейк приготовлен с лёгкими соусами или маринадами, которые могут гармонировать с более деликатными винами.

При выборе вина также стоит учитывать личные предпочтения и то, какие вкусы вы хотите подчеркнуть в своём блюде. Не бойтесь экспериментировать и находить свои идеальные сочетания!

## Assistant API

Создадим контекст диалога и основные установки ассистента

In [115]:
def create_thread():
    return sdk.threads.create(ttl_days=1, expiration_policy='static')

def create_assistant(model, tools=None):
    kwargs = {}
    if tools and len(tools) > 0:
        kwargs = {"tools": tools}
    return sdk.assistants.create(model, ttl_days=1, expiration_policy="since_last_active", **kwargs)

Создаем простого ассистента и беседу:

In [116]:
thread = create_thread()
assistant = create_assistant(model)

assistant.update(instruction="Ты - опытный сомелье, задача которого - консультировать пользователя в вопросах выбора вина.")

thread.write('Привет! Какое вино посоветуете?')

run = assistant.run(thread)
result = run.wait()

printx(result.text)

Здравствуйте! Чтобы подобрать для вас вино, мне нужно узнать немного больше. Расскажите, пожалуйста, предпочитаете ли вы белое, розовое или красное вино? Есть ли у вас любимые сорта винограда или страны-производители? Какой уровень сладости и крепости вам больше по душе? И, возможно, к какому блюду вы планируете подобрать вино?

In [117]:
thread.write("Я буду есть стейк!")

run = assistant.run(thread)
result = run.wait()

printx(result.text)

Для стейка я бы порекомендовал красное вино с хорошим уровнем танинов и средней или высокой крепостью. Например, можно обратить внимание на вина из сортов винограда каберне совиньон, мерло или шираз. Отлично подойдут вина из регионов Бордо, Тоскана или долины Напа. Эти вина хорошо подчеркнут вкус мяса и сбалансируют его текстуру. Если вы предпочитаете более насыщенные и плотные вина, обратите внимание на испанские риохи или аргентинские мальбеки.

Выведем историю диалога:

In [118]:
for msg in list(thread)[::-1]:
  printx(f"**{msg.author.role}:** {msg.text}")

**USER:** Привет! Какое вино посоветуете?

**ASSISTANT:** Здравствуйте! Чтобы подобрать для вас вино, мне нужно узнать немного больше. Расскажите, пожалуйста, предпочитаете ли вы белое, розовое или красное вино? Есть ли у вас любимые сорта винограда или страны-производители? Какой уровень сладости и крепости вам больше по душе? И, возможно, к какому блюду вы планируете подобрать вино?

**USER:** Я буду есть стейк!

**ASSISTANT:** Для стейка я бы порекомендовал красное вино с хорошим уровнем танинов и средней или высокой крепостью. Например, можно обратить внимание на вина из сортов винограда каберне совиньон, мерло или шираз. Отлично подойдут вина из регионов Бордо, Тоскана или долины Напа. Эти вина хорошо подчеркнут вкус мяса и сбалансируют его текстуру. Если вы предпочитаете более насыщенные и плотные вина, обратите внимание на испанские риохи или аргентинские мальбеки.

Можно удалить переписку и ассистента:

In [119]:
thread.delete()
assistant.delete()

## Добавляем RAG

Для RAG будет использоваться база знаний в data/wines и data/regions. Посмотрим на длину файлов в токенах.

In [12]:
from glob import glob
from tqdm.auto import tqdm
import pandas as pd

def get_token_count(filename):
  with open(filename, 'r', encoding='utf-8') as f:
    return len(model.tokenize(f.read()))
  
def get_file_len(filename):
  with open(filename, 'r', encoding='utf-8') as f:
    return len(f.read())
  
d = [
  {
    "File": fn,
    "Tokens": get_token_count(fn),
    "Chars": get_file_len(fn),
    "Category": fn.split('\\')[1],
  }
  for fn in glob("data/*/*.md")
  if fn.count('\\') == 2
]

df = pd.DataFrame(d)
df

c:\Users\Vladislav\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,File,Tokens,Chars,Category
0,data\regions\Абруццо.md,498,2022,regions
1,data\regions\Азорские острова.md,408,1809,regions
2,data\regions\Аконкагуа.md,277,1182,regions
3,data\regions\Алентежу.md,319,1216,regions
4,data\regions\Апулия.md,488,1918,regions
...,...,...,...,...
125,data\wines\Совиньон блан.md,633,2578,wines
126,data\wines\Темпранильо.md,636,2323,wines
127,data\wines\Цвайгельт.md,662,2620,wines
128,data\wines\Шардоне.md,566,2407,wines


Посмотрим минимальную, среднюю и максимальную длину фрагментов:

In [13]:
df.groupby("Category").agg({"Tokens": ("min", "mean", "max")})

Tokens                 
            min        mean  max
Category                        
regions     152  422.340000  682
wines       263  550.366667  663

Загрузим файлы в облако

In [14]:
def upload_file(filename):
  return sdk.files.upload(filename, ttl_days=1, expiration_policy="static")

df["Uploaded"] = df["File"].apply(upload_file)

## Строим индекс

Поскольку есть ограничение на 100 добавляемых в индекс файлов, то будем добавлять фрагменты по винам и регионам по очереди:

In [15]:
from yandex_cloud_ml_sdk.search_indexes import (StaticIndexChunkingStrategy, HybridSearchIndexType, ReciprocalRankFusionIndexCombinationStrategy)

op = sdk.search_indexes.create_deferred(
  df[df["Category"] == "wines"]["Uploaded"],
  index_type = HybridSearchIndexType(
    chunking_strategy=StaticIndexChunkingStrategy(
      max_chunk_size_tokens=1000, chunk_overlap_tokens=100
    ),
    combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
  ),
)

index = op.wait()

In [16]:
op = index.add_files_deferred(df[df["Category"] == "regions"]["Uploaded"])
xfiles = op.wait()

## Собираем RAG-ассистента

Собираем нашего ассистента, который будет использовать RAG. Для этого определим инструмент для поиска в нашем индексе, и указываем его при создании ассистента.

In [17]:
search_tool = sdk.tools.search_index(index)
assistant = create_assistant(model, tools=[search_tool])

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию и
выдай одну или несколько лучших рекомендаций. Если что-то непонятно, то лучше уточни информацию
у пользователя.
"""

thread = create_thread()
_ = assistant.update(instruction=instruction)

In [18]:
thread.write("Какое вино подходит к стейку?")

run = assistant.run(thread)

result = run.wait()
printx(result.text)

К стейку хорошо подойдут:
* красные южноафриканские вина — они гармонируют с перчёными стейками из говядины и баранины;
* пинотаж — универсальное вино, которое подойдёт и к сочному стейку;
* красные вина из региона Бьянкелло дель Метауро — они составят гармоничную пару сочному стейку;
* техасские полнотелые красные вина (темпранильо, каберне совиньон, мерло) — они сочетаются с мясными блюдами;
* красные вина из Кампании — смягчённая выдержкой бархатная танинность таких вин гармонично дополнит вкус сочного стейка.

Посмотрим откуда был получен ответ:

In [19]:
def print_citations(result):
  for citation in result.citations:
    for source in citation.sources:
      if source.type != 'filechunk':
        continue
      print('#'*80)
      printx(source.parts[0])

print_citations(result)

################################################################################



Белые вина Франшхука — прекрасный аперитив. Они подходят к легким сырам, закускам из гусиного паштета и курицы. Среди горячих блюд стоит выбрать индейку, запеченную белую рыбу с овощным гарниром или рагу. 
Красные южноафриканские сорта хорошо гармонируют с перчеными стейками из говядины и баранины, а также твердыми, выдержанными сырами. Пинотаж при этом считается универсальным вином. Ему подходят разнообразные блюда от изысканного филе миньон до лазаньи и пиццы с морепродуктами.

################################################################################



Бьянкелло дель метауро подойдет к рыбе, брускеттам с морепродуктами, белому сыровяленному мясу или любым оливкам.
Красные вина региона составят гармоничную пару сыровяленым колбасам, прошутто и сочному стейку.

################################################################################



Техасские вина очень зависят от терруара. В штате производят полнотелые красные темпранильо, каберне совиньон, мерло, белые из сорта вионье, сухое и полусладкое вино из блан дю буа.
Полнотелые красные вина сочетаются с мясными блюдами, твердыми сырами. Белые подают к рыбе, морепродуктам. Крепленые в стиле портвейна хороши с дичью, выдержанными сырами, орехами, шоколадом.

################################################################################



Красные вина идеально подойдут к любому мясу — как к жирной свинине, так и к нежному каре ягненка или пикантной говядине с ягодным соусом. Белые вина хороши в паре с рыбными блюдами или пастой с морепродуктами.

Сухие и полусладкие мускаты насыщенны и ароматны, поэтому гармонично сочетаются с птицей, выпечкой и даже с луковым супом. И практически ко всем местным винам подойдут твердый сыр грана падано или местный канестрато из козьего и овечьего молока.

################################################################################



Красные вина из Кампании идеально подойдут к пряными закускам с выдержанными сырами — например, к тортилье с грана падана. Смягченная выдержкой бархатная танинность гармонично дополнит и вкус сочного стейка или классической неаполитанской лазаньи.
Белые вина Кампании практически всегда сойдутся с привычными для региона блюдами из рыбы и морепродуктов — это может быть легкий салат, ризотто или паста, а также любые блюда с типичным местным соусом маринара.

Чтобы улучшить ответы по сочетанию вин и еды, добавим таблицу соответствий.

In [20]:
thread.delete()

## Добавление таблицы соответствий

Добавим явную таблицу соответствий еды и вина:

In [21]:
with open("data/food_wine_table.md", encoding='utf-8') as f:
  food_wine = f.readlines()
fw = "".join(food_wine)

tokens = len(model.tokenize(fw))
print(f"Токенов: {tokens}, {len(fw)/tokens} chars/token")

Токенов: 12629, 3.3106342544936256 chars/token


In [22]:
printx(fw[:1000])

Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Баклажаны, запеченые с сыром | Красное вино: «среднетелые»* сухие — Гренаш (Гарнача), Санджовезе (Кьянти), Карменер, Менсия, молодые Темпранильо, легкотелое Мерло.
Баранина деликатесная (филе или каре ягненка) | Красное вино: сухие выдержанные вина из винограда Пино Нуар, Менсия, Неббиоло (в том числе элегантные выдержанные Бароло и Барбареско), Гамэ (элегантные бургундские Божоле Виляж).
Баранина пикантная: жареная, гриль, тушеная — со специями | Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России.
Бефстроганов | Белые вина: выдержанные в дубе Шардоне, Пино Гриджо (лучше — из Северной Италии), Вердехо, Вермент

Чанкуем таблицу соотвествий так как она большая, но так, чтобы в каждом фрагменте был заголовок таблицы:

In [23]:
header = food_wine[:2]
header

['Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду\n',
 '--------|--------\n']

Далее чанкуем таблицу, загружая чанки в облако:

In [24]:
chunk_size = 600 * 3

s = header.copy()
uploaded_foodwine = []
for x in food_wine[2:]:
  s.append(x)
  if len("".join(s)) > chunk_size:
    id = sdk.files.upload_bytes(
      "".join(s).encode(), ttl_days=5, expiration_policy="static", mime_type="text/markdown",
    )
    uploaded_foodwine.append(id)
    s = header.copy()

print(f"Загружено {len(uploaded_foodwine)} чанков")
print(uploaded_foodwine)

Загружено 22 чанков
[File(id='fvt7sghi4nv9gim0ib61', expiration_config=ExpirationConfig(ttl_days=5, expiration_policy=<ExpirationPolicy.STATIC: 1>), name=None, description=None, mime_type='text/markdown', created_by='ajekkr1ccolqr0r204kr', created_at=datetime.datetime(2026, 1, 16, 15, 44, 28, 247748), updated_by='ajekkr1ccolqr0r204kr', updated_at=datetime.datetime(2026, 1, 16, 15, 44, 28, 247748), expires_at=datetime.datetime(2026, 1, 21, 15, 44, 28, 247748), labels=None), File(id='fvtpp1ttkt3mmpkn30pn', expiration_config=ExpirationConfig(ttl_days=5, expiration_policy=<ExpirationPolicy.STATIC: 1>), name=None, description=None, mime_type='text/markdown', created_by='ajekkr1ccolqr0r204kr', created_at=datetime.datetime(2026, 1, 16, 15, 44, 28, 377356), updated_by='ajekkr1ccolqr0r204kr', updated_at=datetime.datetime(2026, 1, 16, 15, 44, 28, 377356), expires_at=datetime.datetime(2026, 1, 21, 15, 44, 28, 377356), labels=None), File(id='fvtqaumicv27dsa47d6c', expiration_config=ExpirationConfi

Теперь добавим эти фрагменты в индекс

In [25]:
op = index.add_files_deferred(uploaded_foodwine)
xfiles = op.wait()

Проверим новый ответ:

In [26]:
thread = create_thread()
thread.write("Какое вино подходит к стейку?")

run = assistant.run(thread)

result = run.wait()
printx(result.text)
print_citations(result)

Выбор вина к стейку зависит от его вида и степени прожарки.

**Для нежного мраморного стейка (филе-миньон)** подойдут:
* лёгкие и элегантные красные вина из винограда Пино Нуар, Нерелло Маскалезе;
* элегантное и выдержанное Мерло;
* «округлые», выдержанные вина из сортов Неббиоло (например, Барбареско), Темпранильо (например, Рибейра дель Дуэро), Санджовезе (например, Кьянти Ризерва).

**Для жирноватого мраморного стейка (рибай и прочие):**
* при прожарке Rare — выдержанные и «благородные» вина из Темпранильо (например, Рибейра дель Дуэро или вина уровня Ризерва и выше), Санджовезе (например, Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки;
* при прожарке Medium или WellDone — сухие и полусухие вина из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш + Сира + Мурведр, вина Приората от 6–8 лет выдержки и выше.

Также можно рассмотреть красные южноафриканские вина, которые хорошо гармонируют с перчёными стейками из говядины, или красные вина из Кампании, которые подойдут к сочному стейку.

################################################################################



Бьянкелло дель метауро подойдет к рыбе, брускеттам с морепродуктами, белому сыровяленному мясу или любым оливкам.
Красные вина региона составят гармоничную пару сыровяленым колбасам, прошутто и сочному стейку.

################################################################################



Стейк говяжий мраморный нежный (Филе-миньон) | Красные вина: легкие и элегантные из винограда Пино Нуар, Нерелло Маскалезе, элегантно сделанное и выдержанное Мерло. Также подойдут «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).
Стейк говяжий мраморный жирноватый (Рибай и пр.) | Красные вина: к прожарке Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки. К прожарке Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6-8 лет выдержки и выше.
Суши, сашими | см. Роллы

...


Свинина стейк жареный с луком | Красные вина: сухие и полусухие вина из винограда Гарнача (Гренаш), Мерло, Карменер, Менсия, «тельные» Пино Нуары (со всего света), российский Красностоп, Гамэ (Божоле Виляж).
Сельдь с лучком (закуска) | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Для глобальных приверженцев вина — тельный, сладковатый и мощновкусный Рислинг.

################################################################################



Белые вина Франшхука — прекрасный аперитив. Они подходят к легким сырам, закускам из гусиного паштета и курицы. Среди горячих блюд стоит выбрать индейку, запеченную белую рыбу с овощным гарниром или рагу. 
Красные южноафриканские сорта хорошо гармонируют с перчеными стейками из говядины и баранины, а также твердыми, выдержанными сырами. Пинотаж при этом считается универсальным вином. Ему подходят разнообразные блюда от изысканного филе миньон до лазаньи и пиццы с морепродуктами.

################################################################################



Красные вина из Кампании идеально подойдут к пряными закускам с выдержанными сырами — например, к тортилье с грана падана. Смягченная выдержкой бархатная танинность гармонично дополнит и вкус сочного стейка или классической неаполитанской лазаньи.
Белые вина Кампании практически всегда сойдутся с привычными для региона блюдами из рыбы и морепродуктов — это может быть легкий салат, ризотто или паста, а также любые блюда с типичным местным соусом маринара.

In [51]:
thread.delete()
assistant.delete()

## Function calling

Делаем ассистента для магазина вин, возьмем пример прайс листа:

In [29]:
import pandas as pd

pl = pd.read_excel("data/wine-price.xlsx")
pl

,Номер артикула,название,CT,цена от 1 бутылки,от 3-х и более,регулярная цена/промо
0,56885,"0,75Л ВИНО САССИКАЙЯ КР СХ",IT,27799.000,19459.3000,NaN
1,666560,"0,75ВИНО СИЕПИ МАЗЕЙ КР СХ",IT,15999.000,11199.3000,NaN
2,533769,"0,75ВИНО ПАЛАФРЕНО КР СХ",IT,14999.004,10499.3028,NaN
3,93733,"0,75ВИНО АНТ ТИНЬЯНЕЛЛО КР СХ",IT,14499.012,10149.3084,NaN
4,644863,"0,75ВИНО ШАТО МОНРОЗ КР СХ",FR,12999.000,9099.3000,от промо цены
...,...,...,...,...,...,...
747,61418,"0,7ВИНО КАГОР ТАМ КР СЛ",RU,179.004,125.3028,от промо цены
748,615581,"0,187ВИНО ДЖАСТ МЕРЛО КР СХ",FR,149.004,104.3028,NaN
749,615582,"0,187ВИНО ДЖАСТ КБСВ КР СХ",FR,149.004,104.3028,NaN
750,83302,"0,187Л ВИНО АДАГУМ КБСВ КР СХ",RU,119.004,83.3028,NaN


Для удобства переименуем колонки:

In [33]:
pl.columns = ["Id", "Name", "Country", "Price", "WHPrice", "etc"]
pl

,Id,Name,Country,Price,WHPrice,etc
0,56885,"0,75Л ВИНО САССИКАЙЯ КР СХ",IT,27799.000,19459.3000,NaN
1,666560,"0,75ВИНО СИЕПИ МАЗЕЙ КР СХ",IT,15999.000,11199.3000,NaN
2,533769,"0,75ВИНО ПАЛАФРЕНО КР СХ",IT,14999.004,10499.3028,NaN
3,93733,"0,75ВИНО АНТ ТИНЬЯНЕЛЛО КР СХ",IT,14499.012,10149.3084,NaN
4,644863,"0,75ВИНО ШАТО МОНРОЗ КР СХ",FR,12999.000,9099.3000,от промо цены
...,...,...,...,...,...,...
747,61418,"0,7ВИНО КАГОР ТАМ КР СЛ",RU,179.004,125.3028,от промо цены
748,615581,"0,187ВИНО ДЖАСТ МЕРЛО КР СХ",FR,149.004,104.3028,NaN
749,615582,"0,187ВИНО ДЖАСТ КБСВ КР СХ",FR,149.004,104.3028,NaN
750,83302,"0,187Л ВИНО АДАГУМ КБСВ КР СХ",RU,119.004,83.3028,NaN


Из имение извлечем кислотность вина:

In [34]:
acid_map = {"СХ": "Сухое", "СЛ": "Сладкое", "ПСХ": "Полусухое", "ПСЛ": "Полусладкое"}
pl["Acidity"] = pl["Name"].apply(
  lambda x: acid_map.get(x.split()[-1].replace("КР", ""), "")
)

pl["Acidity"].value_counts()

Acidity
Сухое          614
Полусухое       68
Полусладкое     28
Сладкое         23
                19
Name: count, dtype: int64

То же самое сделаем с цветом вин:

In [36]:
pl["Color"] = pl["Name"].apply(
  lambda x: (
    'Красное' if (x.split()[-1].startswith("КР") or x.split()[-2] == "КР") else "Белое"
  )
)

pl["Color"].value_counts()

Color
Красное    739
Белое       13
Name: count, dtype: int64

По итогу получили такую таблицу вин

In [37]:
pl

,Id,Name,Country,Price,WHPrice,etc,Acidity,Color
0,56885,"0,75Л ВИНО САССИКАЙЯ КР СХ",IT,27799.000,19459.3000,NaN,Сухое,Красное
1,666560,"0,75ВИНО СИЕПИ МАЗЕЙ КР СХ",IT,15999.000,11199.3000,NaN,Сухое,Красное
2,533769,"0,75ВИНО ПАЛАФРЕНО КР СХ",IT,14999.004,10499.3028,NaN,Сухое,Красное
3,93733,"0,75ВИНО АНТ ТИНЬЯНЕЛЛО КР СХ",IT,14499.012,10149.3084,NaN,Сухое,Красное
4,644863,"0,75ВИНО ШАТО МОНРОЗ КР СХ",FR,12999.000,9099.3000,от промо цены,Сухое,Красное
...,...,...,...,...,...,...,...,...
747,61418,"0,7ВИНО КАГОР ТАМ КР СЛ",RU,179.004,125.3028,от промо цены,Сладкое,Красное
748,615581,"0,187ВИНО ДЖАСТ МЕРЛО КР СХ",FR,149.004,104.3028,NaN,Сухое,Красное
749,615582,"0,187ВИНО ДЖАСТ КБСВ КР СХ",FR,149.004,104.3028,NaN,Сухое,Красное
750,83302,"0,187Л ВИНО АДАГУМ КБСВ КР СХ",RU,119.004,83.3028,NaN,Сухое,Красное


Чтобы Function Calling работал - нам надо сообщить LLM о доступных инструментах. Это можно сделать, передав с помощью JSON-схемы описание возможностей таких инструментов и их параметров.

ML SDK позволяет нам упростить Function Calling, и вместо JSON схемы использовать типизированные объекты Python. Для извлечения параметров запроса о вине, мы создадим такой объект:

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional

class SearchWinePriceList(BaseModel):
  """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""
  
  name: str = Field(description="Название вина", default=None)
  country: str = Field(description="Страна", default=None)
  acidity: str = Field(description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None)
  color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
  sort_order: str = Field(description="Порядок выдачи (most expensive, cheapest, random, average)", default=None)
  what_to_return: str = Field(description="Что вернуть (wine info или price)", default=None)

Теперь создадим инструмент и нового ассистента, у которого в списке инструментов будет одновременно и RAG-поиск, и function calling. Также в инструкции ассистенту пропишем, что он может использовать Function Calling.

In [52]:
price_list_search_tool = sdk.tools.function(SearchWinePriceList)

assistant = create_assistant(model, tools=[price_list_search_tool])
thread = create_thread()

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. Посмотри на всю имеющуюся в твоем распоряжении информацию и
выдай одну или несколько лучших рекомендаций. Если вопрос касается конкретных вин или цены, то используй Function Calling.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

_ = assistant.update(instruction=instruction)

Попробуем узнать самое дешевое вино из Австралии:

In [56]:
thread.write("Привет! Какое есть самое дешевое красное сухое вино из Австралии?")

run = assistant.run(thread)

result = run.wait()
result

RunResult(status=<RunStatus.TOOL_CALLS: 5>, error=None, tool_calls=ToolCallList(tool_calls=(ToolCall(id=None, function=FunctionCall(name='SearchWinePriceList', arguments={'what_to_return': 'wine info', 'acidity': 'сухое', 'color': 'красное', 'sort_order': 'cheapest', 'country': 'Австралия'})),)), _message=None, usage=Usage(input_text_tokens=359, total_tokens=400, completion_tokens=41))

Реализуем функцию, которая возвращает список вин по параметрам, заданным в виде объекта SearchWinePriceList:

In [54]:
country_map = {
    "IT": "Италия",
    "FR": "Франция",
    "ES": "Испания",
    "RU": "Россия",
    "PT": "Португалия",
    "AR": "Армения",
    "CL": "Чили",
    "AU": "Австрия",
    "GE": "Грузия",
    "ZA": "ЮАР",
    "US": "США",
    "NZ": "Новая Зеландия",
    "DE": "Германия",
    "AT": "Австрия",
    "IL": "Израиль",
    "BG": "Болгария",
    "GR": "Греция",
    "AU": "Австралия",
}

revmap = {v.lower(): k for k, v in country_map.items()}


def find_wines(req):
    x = pl.copy()
    if req.country and req.country.lower() in revmap.keys():
        x = x[x["Country"] == revmap[req.country.lower()]]
    if req.acidity:
        x = x[x["Acidity"] == req.acidity.capitalize()]
    if req.color:
        x = x[x["Color"] == req.color.capitalize()]
    if req.name:
        x = x[x["Name"].apply(lambda x: req.name.lower() in x.lower())]
    if req.sort_order and len(x)>0:
        if req.sort_order == "cheapest":
            x = x.sort_values(by="Price")
        elif req.sort_order == "most expensive":
            x = x.sort_values(by="Price", ascending=False)
        else:
            pass
    if x is None or len(x) == 0:
        return "Подходящих вин не найдено"
    return "Вот какие вина были найдены:\n" + "\n".join(
        [
            f"{z['Name']} ({country_map.get(z['Country'],'Неизвестно')}) - {z['Price']}"
            for _, z in x.head(10).iterrows()
        ]
    )


print(find_wines(SearchWinePriceList(country="Австралия", sort_order="cheapest")))

Вот какие вина были найдены:
0,75ВИНО ДЖИНДАЛИ КБСВ КР ПСХ (Австралия) - 499.0
0,75ВИНО ДЖИНДАЛИ МЕРЛО КР ПСХ (Австралия) - 499.0
0,75ВИНО ЧОЛК ХИЛЛ ШИРАЗ КР СХ (Австралия) - 509.0
0,75ВИНО ПИТ'С ПЮР ПННР КР ПСХ (Австралия) - 579.0
0,75ВИНО ПИТ'С ПЮР ШИРАЗ КР ПСХ (Австралия) - 579.0
0,75ВИНО СТАМП ДЖАМП КР СХ (Австралия) - 789.0
0,75ВИНО ЛИНД БИН50 ШИР КР ПСХ (Австралия) - 899.0
0,75ВИНО ЛЭКИ ШИРАЗ КРСХ (Австралия) - 978.996
0,75ВИНО СТЭДФАСТ ШИР КАБ КРСХ (Австралия) - 999.0
0,75ВИНО ТИРРЕЛЗ ШИР КР СХ (Австралия) - 1098.996


Важно выполнить эту ячейку не позднее, чем через 30 секунд после запроса, иначе будет ошибка

In [ ]:
if result.tool_calls:
  res = []
  for f in result.tool_calls:
    # print(f" + Обработка {f.function.name}")
    x = SearchWinePriceList.model_validate(f.function.arguments)
    x = find_wines(x)
    res.append({"name": f.function.name, "content": x})
  run.submit_tool_results(res)
  result = run.wait()

result

 + Обработка SearchWinePriceList


RunResult(status=<RunStatus.COMPLETED: 4>, error=None, tool_calls=None, _message=Message(id='fvtvpptqr50ss845u9pl', parts=('Самое дешёвое красное сухое вино из Австралии в нашем магазине — это «ЧОЛК ХИЛЛ ШИРАЗ» по цене 509 рублей за бутылку объёмом 0,75 литра.',), thread_id='fvtim6tq9c97cnuha0i5', created_by='ajekkr1ccolqr0r204kr', created_at=datetime.datetime(2026, 1, 16, 16, 27, 58, 718044), labels=None, author=Author(id='fvt93re633sa4hf8vn92', role='ASSISTANT'), citations=(), status=<MessageStatus.COMPLETED: 1>), usage=Usage(input_text_tokens=1049, total_tokens=1131, completion_tokens=82))

In [58]:
thread.delete()
assistant.delete()

## Реализуем агента с Function Calling

Для упрощения реализации Function Calling напишем небольшую обвязку, реализующую агента, способного искать в текстовой базе и делать Function Calling.

Функцию обработки запроса мы включим в состав класса для описания функции, назовем ее process. Для реализации всех наших задумок также будем передавать в нее текуший thread:

In [61]:
class SearchWinePriceList(BaseModel):
  """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""
  
  name: str = Field(description="Название вина", default=None)
  country: str = Field(description="Страна", default=None)
  acidity: str = Field(description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None)
  color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
  sort_order: str = Field(description="Порядок выдачи (most expensive, cheapest, random, average)", default=None)
  what_to_return: str = Field(description="Что вернуть (wine info или price)", default=None)

  def process(self, thread):
    return find_wines(self)

Также в виде функции реализуем функционально передачи управления оператору. В данном случае мы будем просто устанавливать некоторый глобальный флаг handover.

In [62]:
handover = False

class Handover(BaseModel):
  """Эта функция позволяет передать диалог человеку-оператору поддержки"""

  reason: str = Field(
    description="Причина для вызова оператора", default="не указана"
  )

  def process(self, thread):
    global handover
    handover = True
    return f"Я побежал вызывать оператора, ваш {thread.id=}, причина: {self.reason}"

Также реализуем функцию добавления вин в корзину. Чтобы для каждого пользователя была своя корзина, будем привязывать ее к thread.id с помощью словаря carts:

In [63]:
carts = {}

class AddToCart(BaseModel):
  """Эта функция позволяет положить или добавить вино в корзину"""

  wine_name: str = Field(
    description="Точное название вина, чтобы положить в корзину", default=None
  )
  count: int = Field(
    description="Количество бутылок вина, которое нужно положить в корзину", default=1
  )

  def process(self, thread):
    if thread.id not in carts:
      carts[thread.id] = []
    carts[thread.id].append(self)
    return f"Вино {self.wine_name} добавлено в корзину, число бутылок: {self.count}"

Создадим функцию для показа корзины:

In [64]:
class ShowCart(BaseModel):
  """Эта функция позволяет показать содержимое корзины"""

  def process(self, thread):
    if thread.id not in carts or len(carts[thread.id]) == 0:
      return "Корзина пуста"
    return "В корзине находятся следующие вина:\n" + "\n".join(
      [f"{x.wine_name}, число бутылок: {x.count}" for x in carts[thread.id]]
    )

Теперь реализуем главный класс Agent, который будет брать на себя обработку функций. В качестве tools будем передавать список описанных нами ранее классов. 

Предусмотрим гибкую обработку thread. При запросе агента мы сможем опционально указывать ему уже созданный thread для ведения переписки, либо же переписка будет вестись в созданном потоке по умолчанию.

In [ ]:
class Agent:
  def __init__(self, assistant=None, instruction=None, search_index=None, tools=None):

    self.thread = None

    if assistant:
      self.assistant = assistant
    else:
      if tools:
        self.tools = {x.__name__: x for x in tools}
        tools = [sdk.tools.function(x) for x in tools]
      else:
        self.tools = {}
        tools = []
      if search_index:
        tools.append(sdk.tools.search_index(search_index))
      self.assistant = create_assistant(model, tools)

    if instruction:
      self.assistant.update(instruction=instruction)

  def get_thread(self, thread=None):
    if thread is not None:
      return thread
    if self.thread == None:
      self.thread = create_thread()
    return self.thread
  
  def __call__(self, message, thread=None):
    thread = self.get_thread(thread)
    thread.write(message)
    run = self.assistant.run(thread)
    res = run.wait()

    if res.tool_calls:
      result = []
      for f in res.tool_calls:
        print(f" + Вызываем функцию {f.function.name}, args={f.function.arguments}")
        fn = self.tools[f.function.name]
        obj = fn(**f.function.arguments)
        x = obj.process(thread)
        result.append({"name": f.function.name, "content": x})
      run.submit_tool_results(result)

      res = run.wait()

    return res.text
  
  def restart(self):
    if self.thread:
      self.thread.delete()
      self.thread = sdk.threads.create(
        name="Test", ttl_days=1, expiration_policy="static"
      )

  def done(self, delete_assistant=False):
    if self.thread:
      self.thread.delete()
    if delete_assistant:
      self.assistant.delete()

Создадим нашего агента - винного сомелье. В системном промпте пропишем ему варианты вызова функции, чтобы облегчить задачу.

In [97]:
instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. Посмотри на всю имеющуюся в твоем распоряжении информацию и
выдай одну или несколько лучших рекомендаций. Рекомендуй конкретные названия вин.
Если вопрос касается конкретных вин или цены, то вызови функцию SearchWinePriceList.
Для передачи управления оператору - вызови функцию Handover. Для добавления вина в корзину используй AddToCart. Для просмотра корзины: ShowCart.
Все названия вин, цветов, кислотности пиши на русском языке.
Если что-то непонятно, то лучше уточни информацию у пользователя. После успешного подбора вина, добавления в корзину и других действий, описанных выше
ответь пользователю, что ты выполнил.
"""

wine_agent = Agent(
  instruction=instruction,
  search_index=index,
  tools=[SearchWinePriceList, Handover, AddToCart, ShowCart]
)

In [67]:
printx(wine_agent('Какое вино пьют со стейком?'))

Выбор вина к стейку зависит от вида мяса и степени его прожарки. Вот несколько рекомендаций:

1. **Для жареного свиного стейка с луком** подойдут:
   - сухие и полусухие вина из винограда Гарнача (Гренаш);
   - Мерло;
   - Карменер;
   - Менсия;
   - «тельные» Пино Нуары (со всего света);
   - российский Красностоп;
   - Гамэ (Божоле Виляж).

2. **Для мраморного нежного говяжьего стейка (филе-миньон)** рекомендуются:
   - лёгкие и элегантные красные вина из Пино Нуар;
   - Нерелло Маскалезе;
   - выдержанное Мерло;
   - «округлые», выдержанные вина из Неббиоло (например, Барбареско);
   - Темпранильо (например, из Рибейра дель Дуэро);
   - Санджовезе (например, Кьянти Ризерва).

3. **Для жирноватого мраморного говяжьего стейка (рибай и пр.)**:
   - к прожарке Rare — выдержанные и «благородные» вина из Темпранильо (например, из Рибейра дель Дуэро или Ризервы);
   - Санджовезе (например, Кьянти Ризерва, Брунелло);
   - «супертосканские» вина;
   - Бордо Правого берега;
   - шелковистые аргентинские Мальбеки.
   - к прожарке Medium или WellDone — сухие и полусухие вина из Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6–8 лет выдержки и выше.

Если вас интересуют конкретные вина из нашего прайс-листа, пожалуйста, сообщите, и я помогу найти подходящие варианты.

In [68]:
printx(wine_agent('Какие вина Кьянти есть в продаже?'))

 + Вызываем функцию SearchWinePriceList, args={'what_to_return': 'wine info', 'name': 'Кьянти'}


В нашем магазине есть следующие вина Кьянти:

1. Кверчаб Кьянти красное сухое (0,75 л) — 2499 рублей.
2. Полиц Кьянти красное сухое (0,75 л) — 1749,76 рублей.
3. Касал Кьянти супер красное сухое (0,75 л) — 1349 рублей.
4. Век Кант Кьянти красное сухое (0,75 л) — 1099 рублей.
5. Пределла Кьянти красное сухое (1,5 л) — 999 рублей.
6. Зонин Кьянти красное сухое (0,75 л) — 699 рублей.
7. Пределла Кьянти красное сухое (0,75 л) — 369 рублей.

Если вам нужно больше информации о каком-либо из этих вин или вы хотите добавить вино в корзину, пожалуйста, сообщите мне.

In [69]:
printx(wine_agent('Добавь в корзину Полиц Кьянти, три бутылки'))

 + Вызываем функцию AddToCart, args={'wine_name': 'Полиц Кьянти', 'count': 3.0}


Три бутылки вина Полиц Кьянти добавлены в вашу корзину. Если вам нужно что-то ещё, пожалуйста, дайте знать!

In [70]:
printx(wine_agent('Еще положи Зонин Кьянти'))

 + Вызываем функцию AddToCart, args={'wine_name': 'Зонин Кьянти', 'count': 1.0}


Вино Зонин Кьянти добавлено в вашу корзину. Если вам нужно что-то ещё, пожалуйста, дайте знать!

In [71]:
printx(wine_agent('Что у меня в корзине?'))

 + Вызываем функцию ShowCart, args={}


В вашей корзине:
- Полиц Кьянти — 3 бутылки;
- Зонин Кьянти — 1 бутылка.

Если вам нужно что-то изменить в корзине или нужна дополнительная помощь, пожалуйста, сообщите!

In [95]:
printx(wine_agent('давайте возьмём Мерло, мне больше мягкий вкус нравится. Найдите самое недорогое Мерло и положите в корзину.'))

<IPython.core.display.Markdown object>

## Многоагентное тестирование

In [98]:
instruction_user = """
Ты - простой человек, и тебе нужно выбрать вино в интернет-магазине для ужина.
Ты не очень много понимаешь в винах, и хочешь побольше распросить сомелье о разных вариантах.
Ты хотел бы съесть стейк, но готов изменить свой выбор еды, если поймёшь хорошее сочетание с вином. 
Говори простым языком, короткими разговорными фразами. Когда ты удовлетворём выбором, попроси 
найти самое недорогое вино данного типа, затем положи его в корзину. В конце попроси соединить тебя 
с оператором, чтобы оформить доставку. Каждое действие выполняй в одной фразой в ответ на сообщение сомелье. Никогда 
не продолжай диалог, больше, чем одной фразой. Не пиши реплики от лица сомелье или кого-то другого.
"""

user = Agent(instruction=instruction_user)

msg = "Добрый день! Поможете мне выбрать вино?"
handover = False
for i in range(10):
    printx(f"**Посетитель:** {msg}")
    msg = wine_agent(msg)
    printx(f"**Сомелье:** {msg}")
    if handover:
        break
    msg = user(msg)

**Посетитель:** Добрый день! Поможете мне выбрать вино?

**Сомелье:** Здравствуйте! Конечно, помогу! Пожалуйста, расскажите, к какому блюду вы хотите подобрать вино, и я дам вам несколько рекомендаций.

**Посетитель:** Хочу к стейку, но могу и другое блюдо выбрать, если подскажете, с чем вино лучше сочетается. Порекомендуйте пару вариантов красных вин, пожалуйста.

**Сомелье:** Для стейка отлично подойдут красные вина из южноафриканских сортов или Пинотаж — он считается универсальным и подойдёт даже к перчёным стейкам из говядины и баранины.

Если вы готовы рассмотреть другие блюда, вот несколько вариантов, к которым хорошо подойдут красные вина:
* к плову узбекскому — полусухие вина из винограда Каберне-Совиньон и Сира, вина из Роны (Гренаш + Сира + Мурведр), Саперави (Россия), Арени (Армения), Примитиво и Зинфандель;
* к блинчикам с мясом (говядина) — итальянские вина из Санджовезе (Кьянти), «бордосский» бленд, «ронский» ассамбляж Гренаш + Сира + Мурведр.

Могу поискать конкретные вина в нашем прайс-листе, если вас заинтересует какой-то из вариантов.

**Посетитель:** Давайте посмотрим Пинотаж, он вроде универсальный. Найдите самое недорогое вино этого сорта, пожалуйста.

**Сомелье:** Я нашёл для вас вино: «Сурвайвер Пинотаж» из ЮАР. Объём бутылки — 0,75 литра, цена — 1319 рублей. Это самое доступное вино сорта Пинотаж в нашем магазине. Хотите добавить его в корзину?

**Посетитель:** Да, добавьте в корзину. Теперь соедините меня с оператором, пожалуйста, хочу оформить доставку.

**Сомелье:** Вино «Сурвайвер Пинотаж» добавлено в корзину. Сейчас я соединю вас с оператором для оформления доставки. Пожалуйста, ожидайте.

In [ ]:
user.done(delete_assistant=True)
wine_agent.done(delete_assistant=True)

## Делаем ассистента проактивным

In [99]:
instruction = """
Ты - опытный сомелье, продающий вино в магазине. Твоя задача - отвечать на вопросы пользователя
про вина, рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина,
а также проактивно предлагать пользователю приобрести вина, отвечающие его потребностям. В ответ
на сообщение /start поинтересуйся, что нужно пользователю, предложи ему какой-то
интересный вариант сочетания еды и вина, и попытайся продать ему вино.
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию SearchWinePriceList.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке.
Если что-то непонятно, то лучше уточни информацию у пользователя. Общайся достаточно короткими 
разговорными фразами, не используй перечисления, списки, длинные выдержки текста.
После выполненых действий напиши пользователю, что ты выполнил.
"""

wine_agent = Agent(
  instruction=instruction,
  search_index=index,
  tools=[SearchWinePriceList, Handover, AddToCart, ShowCart]
)

In [100]:
printx(wine_agent('/start'))

Здравствуйте! Чем могу помочь? Хотите узнать о винах? Например, белое вино Совиньон Блан отлично подойдёт к морепродуктам — раскрывает их вкус. А может, ищете что-то покрепче? У нас есть насыщенные красные вина из региона Мадиран во Франции. Есть что-то конкретное, что вас интересует?

In [101]:
printx(wine_agent('Сколько стоит самый дорогий Шираз?'))

Самое дорогое вино Шираз в нашем магазине — это «Мантра Шираз» из России. Его цена составляет 1899 рублей за бутылку объёмом 0,75 литра. Могу предложить добавить его в корзину, если вас заинтересовало это вино.

## Делаем винного ассистента в телеграмме

Необходимо создать бота в ТГ с помощью @botfather и разместить его токен в .env файле

In [102]:
%pip install --quiet telebot

Note: you may need to restart the kernel to use updated packages.


In [109]:
import telebot

telegram_token = dotenv_values()['tg_token']

bot = telebot.TeleBot(telegram_token)

threads = {}

def get_thread(chat_id):
  if chat_id in threads.keys():
    return threads[chat_id]
  t = create_thread()
  print(f"Новый тред {t.id=} создан!")
  threads[chat_id] = t
  return t

@bot.message_handler(commands=['start'])
def start(message):
  t = get_thread(message.chat.id)
  print(f"Начало в треде {t.id=}, msg = {message.text}")
  ans = wine_agent(message.text, thread=t)
  bot.send_message(message.chat.id, ans)

@bot.message_handler(func=lambda message: True)
def handle_message(message):
  t = get_thread(message.chat.id)
  print(f"Отвечаю в треде {t.id=}, msg = {message.text}")
  answer = wine_agent(message.text, thread=t)
  bot.send_message(message.chat.id, answer)

print("Бот готов к работе")
bot.polling(non_stop=True)

Бот готов к работе
Новый тред t.id='fvtq31ej1k05vta6mjtc' создан!
Отвечаю в треде t.id='fvtq31ej1k05vta6mjtc', msg = Привет, мне нужно вино
Отвечаю в треде t.id='fvtq31ej1k05vta6mjtc', msg = Хочу самое дешевое красное вино
Отвечаю в треде t.id='fvtq31ej1k05vta6mjtc', msg = Добавь авторское в корзину, три штуки
Отвечаю в треде t.id='fvtq31ej1k05vta6mjtc', msg = Позови оператора, чтобы сделать заказ
